<a href="https://colab.research.google.com/github/quetion/Basic-Python-Programming/blob/master/VIX_construct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# output setting
plt.style.use('seaborn-white')
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
pd.options.display.width = 130

<ipython-input-2-d11939743521>:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
# 設定雲端硬碟中資料夾位置
os.chdir('/content/drive/MyDrive/[0]博士班/112_2/期貨與選擇權_台大/VIX white paper/data')
!ls -l #列出當下資料夾的檔案

total 133603
-rw------- 1 root root     17841 Apr 22 16:36 daily-treasury-rates_2019.csv
-rw------- 1 root root    624582 Apr 24 06:15 IvyDB_US_v5.4_Reference_Manual.pdf
-rw------- 1 root root     39933 Apr 24 08:29 rf_term.zip
-rw------- 1 root root 133525356 Apr 22 16:18 spx_2019.zip
-rw------- 1 root root   2126794 Apr 23 08:03 spx_202302_test.zip
-rw------- 1 root root    100395 Apr 29 05:47 VIX_construct.ipynb
-rw------- 1 root root    372868 Mar 28 05:33 vixts.csv


In [4]:
# import zip-data
dfoptions = pd.read_csv('spx_2019.zip',compression='zip',
                        usecols = ['date','symbol','exdate','cp_flag','strike_price',
                                   'best_bid','best_offer','am_settlement','expiry_indicator'])
# reset the variable
dfoptions['strike_price'] = dfoptions['strike_price']/1000
# reset the date format
dfoptions['date'] = pd.to_datetime(dfoptions['date'],format='%Y-%m-%d')
dfoptions['exdate'] = pd.to_datetime(dfoptions['exdate'],format='%Y-%m-%d')
# check the dfoptions information
#dfoptions.info()

In [ ]:
# import interest rate (from )
#dfrf = pd.read_csv('daily-treasury-rates_2019.csv')
# rename one of columns
#dfrf.rename(columns = {'Date':'date'}, inplace=True)
# reset the date format
#dfrf['date'] = pd.to_datetime(dfrf['date'],format='%m/%d/%Y')
# set 'date' to be the index
#dfrf = dfrf.sort_values('date').set_index('date')
#dfrf = dfrf/100
# reset the index from 0 to n
#dfrf = dfrf.reset_index()

In [ ]:
# SPX's exweekday could be "Friday" or "Thursday"
# In general , they are Friday
# In cases of the Friday being the holiday, the expiration could be Thursday
#aa = SPX['exweekday'].unique()
#SPX[SPX['exweekday'] == aa[1]].head()

In [7]:
# Determine the days of am and pm settlement
# am-settlement : 用開盤價結算
# pm-settlement : 用收盤價結算
dfoptions['days'] = np.where(dfoptions['am_settlement'] == 0,
                             dfoptions['exdate'] - dfoptions['date'],
                             dfoptions['exdate'] - dfoptions['date'] - pd.to_timedelta(1,'days'))
# Determine the weekday in the expiration
dfoptions['exweekday'] = dfoptions['exdate'].dt.day_name()
# Select end-of-week SPXW and copy to dfSPXW
dfSPXW = dfoptions[dfoptions['symbol'].str.contains('SPXW')].copy()
# Select the SPXW of (i) expiration on Friday and (ii) weekly option (expiring at end of week) and (iii) PM-settled
# using "expiry_indicator" to justify the expiration on end of week ('w') or month ('m')
dfSPXW = dfSPXW[(dfSPXW['exweekday'] == 'Friday')&(dfSPXW['expiry_indicator']=='w')&(dfSPXW['am_settlement'] == 0)].copy()
# Combine end-of-week SPXWs with SPXs
dfoptions2 = pd.concat([dfSPXW, dfoptions[~dfoptions['symbol'].str.contains('SPXW')]]).sort_values(['date','days','cp_flag','strike_price'])
dfoptions2.head()

,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,am_settlement,expiry_indicator,days,exweekday
4489,2019-01-02,SPXW 190104C800000,2019-01-04,C,800.0,1703.0,1711.1,0,w,2 days,Friday
4490,2019-01-02,SPXW 190104C900000,2019-01-04,C,900.0,1602.8,1610.6,0,w,2 days,Friday
4244,2019-01-02,SPXW 190104C1000000,2019-01-04,C,1000.0,1502.8,1510.6,0,w,2 days,Friday
4245,2019-01-02,SPXW 190104C1050000,2019-01-04,C,1050.0,1452.9,1460.6,0,w,2 days,Friday
4246,2019-01-02,SPXW 190104C1100000,2019-01-04,C,1100.0,1402.8,1410.6,0,w,2 days,Friday


In [8]:
# A function outputs the near-term and next-term dfs for a given df at a given date
def keepNearNextTermOptions(df):
  maturities = pd.to_timedelta(df['days'].unique(),'days')
  nearT = maturities[maturities <= pd.to_timedelta(30,'days')].max()
  nextT = maturities[maturities > nearT].min()
  df1 = df[df['days'] == nearT].copy()
  df2 = df[df['days'] == nextT].copy()
  return pd.concat([df1,df2])


In [9]:
# Selected optinos are grouped by dates
dfgrouped = dfoptions2.groupby('date')
dfs = []
# For every date, find the near-term and next-term of maturities. Append them to a new df
for date,df in dfgrouped:
  dfnearnext = keepNearNextTermOptions(df)
  dfs.append(dfnearnext)
# the near-term and next-term options have been selelcted
dfoptions3 = pd.concat(dfs)

In [13]:
dfoptions3[dfoptions['date'] == '2019-01-02']


<ipython-input-13-69928cdb5a31>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfoptions3[dfoptions['date'] == '2019-01-02']


,date,symbol,exdate,cp_flag,strike_price,best_bid,best_offer,am_settlement,expiry_indicator,days,exweekday
9370,2019-01-02,SPXW 190201C100000,2019-02-01,C,100.0,2399.0,2406.8,0,w,30 days,Friday
9417,2019-01-02,SPXW 190201C200000,2019-02-01,C,200.0,2299.2,2306.9,0,w,30 days,Friday
9599,2019-01-02,SPXW 190201C300000,2019-02-01,C,300.0,2199.5,2207.3,0,w,30 days,Friday
9615,2019-01-02,SPXW 190201C400000,2019-02-01,C,400.0,2099.6,2107.3,0,w,30 days,Friday
9616,2019-01-02,SPXW 190201C500000,2019-02-01,C,500.0,2000.0,2007.8,0,w,30 days,Friday
...,...,...,...,...,...,...,...,...,...,...,...
10647,2019-01-02,SPXW 190208P2875000,2019-02-08,P,2875.0,362.4,369.9,0,w,37 days,Friday
10648,2019-01-02,SPXW 190208P2900000,2019-02-08,P,2900.0,387.5,394.7,0,w,37 days,Friday
10649,2019-01-02,SPXW 190208P2950000,2019-02-08,P,2950.0,437.2,444.6,0,w,37 days,Friday
10650,2019-01-02,SPXW 190208P3000000,2019-02-08,P,3000.0,486.9,494.2,0,w,37 days,Friday


In [ ]:
# A function applying the above method and the cubic spline method to calculate the riskfree rate
def Riskfree(df, dfrf=dfrf):
  t = df['daynum']
  date = df['date']
  yieldcurve = dfrf[dfrf['date'] == date]
  if t==30:
    BEY = yieldcurve['1 Mo'].squeeze()
  elif (t>30)&(t<60):
    CSinter = CubicSpline([30,60],[yieldcurve['1 Mo'].squeeze(),yieldcurve['2 Mo'].squeeze()],bc_type='natural')
    BEY = float(CSinter(t))
  elif t<30:
    x = 1
    z = 1
    CMT1 = yieldcurve['1 Mo'].squeeze()
    for i in np.arange(2, len(yieldcurve.columns),1):
      if yieldcurve.iloc[0,i]>=CMT1:
        CMTx = yieldcurve.iloc[0,i]
        x = i
        break
    for i in np.arange(2, len(yieldcurve.columns),1):
      if yieldcurve.iloc[0,i]<=CMT1:
        CMTz = yieldcurve.iloc[0,i]
        z = i
        break
    keys = np.arange(2,len(yieldcurve.columns),1)
    values= [60,91,182,365,730,1095,1825,2555,3650,7300,10950]
    d = dict(zip(keys,values))
    if x == 1:
      blower=CMT1
    else:
      blower=CMT1 - ((CMTx-CMT1)/(d[x]-30))*30
    if z == 1:
      bupper= CMT1
    else:
      bupper=CMT1-((CMTz-CMT1)/(d[z]-30))*30
    CSinter = CubicSpline([0,30],[blower,bupper],bc_type='natural')
    BEY=float(CSinter(t))

  APY =((1+BEY/2)**2)-1
  rf = np.log(1+APY)
  return rf

